In [1]:
# IMPORT PACKAGES
import pandas as pd
import numpy as np

# 1. IMPORT USA FACTS CONFIRMED CASES DATA 
# import urllib to pull in the data
from urllib import request
# read the usa facts confirmed cases data it is in UTF8 BOM then convert to UTF8
confirmed_utf8 = request.urlopen("https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv").read().decode("utf-8-sig").encode("utf-8")
# check the data type - make sure it is bytes - not essential for final code
type(confirmed_utf8)
# import StringIO
from io import StringIO
# create dataframe
cases = pd.read_csv(StringIO(str(confirmed_utf8,'utf-8')))

# CREATE A UNIQUE LIST OF 'states' ALONG WITH its 'stateFIPS'
state_fips = cases[['State', 'stateFIPS']].drop_duplicates()

# 2. CASES BY STATES
# group date columns by state using groupby
states_cases = cases.groupby('State', as_index=False).sum()
# create data frame with StateFIPS and State before dropping this for melt
states_cases_fips = states_cases[['State', 'stateFIPS']]
# drop countyFIPS, CountyName and Stat
states_cases_drop = states_cases.drop(['countyFIPS','stateFIPS'], axis=1)
# melt the dataframe by state
states_cases_melt = pd.melt(states_cases_drop, id_vars=['State']).rename(columns={'variable':'date','value':'cases'})
states_cases_melt

# 3. IMPORT USA FACTS CONFIRMED DEATHS FILE 
# read the data it is in UTF8 BOM then convert to UTF8
deaths_utf8 = request.urlopen("https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_deaths_usafacts.csv").read().decode("utf-8-sig").encode("utf-8")
# create dataframe
deaths = pd.read_csv(StringIO(str(deaths_utf8,'utf-8')))

# 4. DEATHS BY STATES
# group date columns by state using groupby
states_deaths = deaths.groupby('State', as_index=False).sum()
# drop countyFIPS, CountyName and Stat
states_deaths_drop = states_deaths.drop(['countyFIPS','stateFIPS'], axis=1)
# melt the dataframe by state
states_deaths_melt = pd.melt(states_deaths_drop, id_vars=['State']).rename(columns={'variable':'date','value':'deaths'})
states_deaths_melt

# 5. MERGE STATE CASES AND DEATHS DATA
merged_cases_deaths = states_cases_melt.merge(states_deaths_melt, on=['State', 'date'])
merged_cases_deaths.tail(5)

# 6. iIMPORT USA FACTS POPULATION FILE  
# read the data it is in UTF8 BOM then convert to UTF8
population_utf8 = request.urlopen("https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_county_population_usafacts.csv").read().decode("utf-8-sig").encode("utf-8")
# create dataframe
population = pd.read_csv(StringIO(str(population_utf8,'utf-8')))
# 7. POPULATION BY STATE
# group by state using groupby
states_pop = population.groupby('State', as_index=False).sum()
# drop countyFIPS, CountyName and Stat
states_pop_drop = states_pop.drop(['countyFIPS'], axis=1)

# 8. MERGE CASES INTO DEATHS INTO POPULATION AND THEN MERGE BACK'stateFIPS'
merged_cases_deaths_pop = merged_cases_deaths.merge(states_pop_drop, on=['State'])
merged_with_pop = merged_cases_deaths_pop.merge(state_fips, on=['State'])

#9. INSERT FULL STATE NAME SO HAVE CHOICE ON USING STATE ABBREVIATION
state_name = pd.DataFrame({'stateFIPS': [1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56], 'state_name': ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']})
states = merged_with_pop.merge(state_name, on=['stateFIPS'])

#10. CREATE DATE TIME FOR CREATING DATA POINTS FOR USE AS A CALCULATED FIELD INSERT IN A TITLE
import datetime
states['date_time'] = pd.to_datetime(states['date'])

# 11. RENAME AND REORDER FOR EASE OF VIEW
states.columns = ['state','date','cases', 'deaths','population','fips', 'state_name', 'date_time']
states = states[['date','date_time','state','state_name','fips','population', 'cases', 'deaths']]

# calculate incremental cases and deaths 
states['new_cases'] = (states.groupby('fips')['cases'].diff().fillna(states['cases'], downcast='infer'))
states['new_deaths'] = (states.groupby('fips')['deaths'].diff().fillna(states['deaths'], downcast='infer'))

# calculate 7 day rolling averages
states['new_cases_7_day_roll_av'] = states.groupby('fips')['new_cases'].transform(lambda x: x.rolling(7, 1).mean().round())
states['new_deaths_7_day_roll_av'] = states.groupby('fips')['new_deaths'].transform(lambda x: x.rolling(7, 1).mean().round())

# 11. CREATE PER 100,000 CALCULATION COLUMNS
# create column for cases per 100,000
states['cases_per_100,000'] = (states['cases']/(states['population']/100000)).round(0).astype(int)
# create column for deaths per 100,000
states['deaths_per_100,000'] = (states['deaths']/(states['population']/100000)).round(0).astype(int)
# create column for new cases per 100,000
states['new_cases_per_100,000'] = (states['new_cases']/(states['population']/100000)).round(0).astype(int)
# create column for new deaths per 100,000
states['new_deaths_per_100,000'] = (states['new_deaths']/(states['population']/100000)).round(0).astype(int)
# create column for new cases seven day rolling average per 100,000
states['new_cases_7_day_roll_av_per_100,000'] = (states['new_cases_7_day_roll_av']/(states['population']/100000)).round(0).astype(int)
# create column for new deaths seven day rolling average per 100,000
states['new_deaths_7_day_roll_av_per_100,000'] = (states['new_deaths_7_day_roll_av']/(states['population']/100000)).round(0).astype(int)

# create 14 day prior comparison for new cases
states['new_cases_14 day_comp']  = (states.groupby('fips')['new_cases_7_day_roll_av'].shift(periods = 14, fill_value = 0))
# calculate the delta for new cases over the prior 14 days 
states['new_cases_14 day_delta'] = states['new_cases_7_day_roll_av']/states['new_cases_14 day_comp']-1

# create columns for ease of inserting data points using calculated fields in Tableau 
states['nj_cases_most_recent'] = states.loc[(states['state'] == 'NJ') & (states['date_time'] == str(states['date_time'].max())), 'cases'].iloc[0]
states['nj_deaths_most_recent'] = states.loc[(states['state'] == 'NJ') & (states['date_time'] == str(states['date_time'].max())), 'deaths'].iloc[0]
states['nj_new_cases_14_day_delta_most_recent'] = states.loc[(states['state'] == 'NJ') & (states['date_time'] == str(states['date_time'].max())), 'new_cases_14 day_delta'].iloc[0]

# EXPORT TO A TEXT FILE
states.to_csv('states.txt', sep='\t')

